The Walmart project had two parts: 1. involved looking at data for various stores, understanding some basic stats like store with max sales, max std. In addition the focus was to figure out the holidays that have a negative impact on sales and provide some insights on monthly and quarterly sales. 2. for store 1: build a linear regression model to hypothesize if CPI, unemployment and fuel prices have an impact on sales. 

There is weekly data here for 45 Walmart stores for a total of 6435 rows. 
Store with Max and Min sales
Store  #20 has the Max sales and store #33 has min sales
Store 14 has max std of 317569.949476

Which store/s had good quarterly growth in 3Q 2012
all stores had negative (year over year) sales growth in 3Q2012: 
The stores with the least negative sales growth were #44(-3.98%),#38(-5.82%),#39(-5.9%)

Comment on store data on a monthly and semester
Retail sales are impacted by seasonality and the economy. 2010 was a recession year and WMT's sales were lower compared to 2011. 
On a monthly basis sales are impacted by timing of holidays and weekends.
Some of the highest months are:
April/May(depending on when Easter falls): sales are impacted by the Easter Holiday and Memorial day. 
Nov/Dec: sales are impacted by the Thanksgiving holiday, Black Friday after Thanksgiving and Christmas.
July and Sept: July by 4th of july holiday and sept is impacted by back to school shopping in addition to holiday

#Linear regression model: The equation using Fuel prices,CPI, unemployment.
#The equation i get is:
# -3887095.77+ -23562.61963268(x1) + 14213.48484339 (x2)+ 67766.41155638 (x3)
#the model doesnt appear to be good with an accuracy score of 7.4%

In [1]:
import scipy
import sklearn 
import datetime
from datetime import datetime
from scipy import stats

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
wmtdata=pd.read_csv('Walmart_Store_sales.csv')

In [4]:
wmtdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [5]:
wmtdata.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

In [6]:
wmtdata.isna()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
6430,False,False,False,False,False,False,False,False
6431,False,False,False,False,False,False,False,False
6432,False,False,False,False,False,False,False,False
6433,False,False,False,False,False,False,False,False


In [7]:
wmtdata.isna().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [8]:
wmtdata.index

RangeIndex(start=0, stop=6435, step=1)

In [9]:
wmtdata.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [10]:
wmtdata.ndim

2

In [11]:
wmtdata.describe

<bound method NDFrame.describe of       Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0         1  05-02-2010    1643690.90             0        42.31       2.572   
1         1  12-02-2010    1641957.44             1        38.51       2.548   
2         1  19-02-2010    1611968.17             0        39.93       2.514   
3         1  26-02-2010    1409727.59             0        46.63       2.561   
4         1  05-03-2010    1554806.68             0        46.50       2.625   
...     ...         ...           ...           ...          ...         ...   
6430     45  28-09-2012     713173.95             0        64.88       3.997   
6431     45  05-10-2012     733455.07             0        64.89       3.985   
6432     45  12-10-2012     734464.36             0        54.47       4.000   
6433     45  19-10-2012     718125.53             0        56.47       3.969   
6434     45  26-10-2012     760281.43             0        58.85       3.882   

     

In [12]:
wmtdata.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [13]:
wmtdata.tail(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
6430,45,28-09-2012,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,05-10-2012,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,12-10-2012,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,19-10-2012,718125.53,0,56.47,3.969,192.330854,8.667
6434,45,26-10-2012,760281.43,0,58.85,3.882,192.308899,8.667


In [14]:
wmtdata.dtypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [15]:
#There is weekly data here for 45 Walmart stores for a total of 6435 rows. 
#Store with Max sales
#Store  #20 has the Max sales and store #33 has min sales

In [16]:
wmt_sales=wmtdata.loc[:,['Store','Weekly_Sales']]

In [17]:
wmt_sales

,Store,Weekly_Sales
0,1,1643690.90
1,1,1641957.44
2,1,1611968.17
3,1,1409727.59
4,1,1554806.68
...,...,...
6430,45,713173.95
6431,45,733455.07
6432,45,734464.36
6433,45,718125.53


In [18]:
sales_bystore=wmt_sales.groupby('Store').sum()

In [19]:
sales_bystore

,Weekly_Sales
Store,
1,2.224028e+08
2,2.753824e+08
3,5.758674e+07
4,2.995440e+08
5,4.547569e+07
6,2.237561e+08
7,8.159828e+07
8,1.299512e+08
9,7.778922e+07


In [20]:
sales_bystore.sort_values(by='Weekly_Sales', ascending=False)

,Weekly_Sales
Store,
20,3.013978e+08
4,2.995440e+08
14,2.889999e+08
13,2.865177e+08
2,2.753824e+08
10,2.716177e+08
27,2.538559e+08
6,2.237561e+08
1,2.224028e+08


In [21]:
min_str_sales=wmtdata.groupby('Store')['Weekly_Sales'].min()

In [22]:
min_str_sales.sort_values(ascending=True)

Store
33     209986.25
44     241937.11
5      260636.71
36     270677.98
38     303908.81
3      339597.38
16     368600.00
30     369722.32
7      372673.61
29     395987.24
42     428953.60
37     451327.61
9      452905.22
15     454183.42
43     505405.85
18     540922.94
25     558794.63
35     576332.05
21     596218.24
45     617207.58
17     635862.55
40     764014.75
8      772539.12
22     774262.28
12     802105.50
26     809833.21
34     836717.75
32     955463.84
41     991941.73
23    1016756.10
24    1057290.41
28    1079669.11
11    1100418.69
39    1158698.44
19    1181204.53
31    1198071.60
6     1261253.18
27    1263534.86
1     1316899.31
14    1479514.66
10    1627707.31
13    1633663.12
2     1650394.44
20    1761016.51
4     1762539.30
Name: Weekly_Sales, dtype: float64

In [23]:
#which store has max standard deviation i.e sales vary a lot,
#find the coefficient of mean to standard deviation
#Store 14 has the highest standard deviation

#mean sales are below; while not asked for is useful information.

In [24]:
mean_str_sales=wmtdata.groupby('Store')['Weekly_Sales'].mean()
mean_str_sales

Store
1     1.555264e+06
2     1.925751e+06
3     4.027044e+05
4     2.094713e+06
5     3.180118e+05
6     1.564728e+06
7     5.706173e+05
8     9.087495e+05
9     5.439806e+05
10    1.899425e+06
11    1.356383e+06
12    1.009002e+06
13    2.003620e+06
14    2.020978e+06
15    6.233125e+05
16    5.192477e+05
17    8.935814e+05
18    1.084718e+06
19    1.444999e+06
20    2.107677e+06
21    7.560691e+05
22    1.028501e+06
23    1.389864e+06
24    1.356755e+06
25    7.067215e+05
26    1.002912e+06
27    1.775216e+06
28    1.323522e+06
29    5.394514e+05
30    4.385796e+05
31    1.395901e+06
32    1.166568e+06
33    2.598617e+05
34    9.667816e+05
35    9.197250e+05
36    3.735120e+05
37    5.189003e+05
38    3.857317e+05
39    1.450668e+06
40    9.641280e+05
41    1.268125e+06
42    5.564039e+05
43    6.333247e+05
44    3.027489e+05
45    7.859814e+05
Name: Weekly_Sales, dtype: float64

In [25]:
mean_str_sales.sort_values(ascending=False)

Store
20    2.107677e+06
4     2.094713e+06
14    2.020978e+06
13    2.003620e+06
2     1.925751e+06
10    1.899425e+06
27    1.775216e+06
6     1.564728e+06
1     1.555264e+06
39    1.450668e+06
19    1.444999e+06
31    1.395901e+06
23    1.389864e+06
24    1.356755e+06
11    1.356383e+06
28    1.323522e+06
41    1.268125e+06
32    1.166568e+06
18    1.084718e+06
22    1.028501e+06
12    1.009002e+06
26    1.002912e+06
34    9.667816e+05
40    9.641280e+05
35    9.197250e+05
8     9.087495e+05
17    8.935814e+05
45    7.859814e+05
21    7.560691e+05
25    7.067215e+05
43    6.333247e+05
15    6.233125e+05
7     5.706173e+05
42    5.564039e+05
9     5.439806e+05
29    5.394514e+05
16    5.192477e+05
37    5.189003e+05
30    4.385796e+05
3     4.027044e+05
38    3.857317e+05
36    3.735120e+05
5     3.180118e+05
44    3.027489e+05
33    2.598617e+05
Name: Weekly_Sales, dtype: float64

In [26]:
#which store has max standard deviation i.e sales vary a lot,
#find the coefficient of mean to standard deviation
#Store 14 has the highest standard deviation

#Also can do std_str_sales=wmtdata.groupby('Store')['Weekly_Sales'].std().max()

In [27]:
std_str_sales=wmtdata.groupby('Store')['Weekly_Sales'].std()


In [28]:
std_str_sales

Store
1     155980.767761
2     237683.694682
3      46319.631557
4     266201.442297
5      37737.965745
6     212525.855862
7     112585.469220
8     106280.829881
9      69028.666585
10    302262.062504
11    165833.887863
12    139166.871880
13    265506.995776
14    317569.949476
15    120538.652043
16     85769.680133
17    112162.936087
18    176641.510839
19    191722.638730
20    275900.562742
21    128752.812853
22    161251.350631
23    249788.038068
24    167745.677567
25    112976.788600
26    110431.288141
27    239930.135688
28    181758.967539
29     99120.136596
30     22809.665590
31    125855.942933
32    138017.252087
33     24132.927322
34    104630.164676
35    211243.457791
36     60725.173579
37     21837.461190
38     42768.169450
39    217466.454833
40    119002.112858
41    187907.162766
42     50262.925530
43     40598.413260
44     24762.832015
45    130168.526635
Name: Weekly_Sales, dtype: float64

In [29]:
std_str_sales.sort_values(ascending=False)

Store
14    317569.949476
10    302262.062504
20    275900.562742
4     266201.442297
13    265506.995776
23    249788.038068
27    239930.135688
2     237683.694682
39    217466.454833
6     212525.855862
35    211243.457791
19    191722.638730
41    187907.162766
28    181758.967539
18    176641.510839
24    167745.677567
11    165833.887863
22    161251.350631
1     155980.767761
12    139166.871880
32    138017.252087
45    130168.526635
21    128752.812853
31    125855.942933
15    120538.652043
40    119002.112858
25    112976.788600
7     112585.469220
17    112162.936087
26    110431.288141
8     106280.829881
34    104630.164676
29     99120.136596
16     85769.680133
9      69028.666585
36     60725.173579
42     50262.925530
3      46319.631557
38     42768.169450
43     40598.413260
5      37737.965745
44     24762.832015
33     24132.927322
30     22809.665590
37     21837.461190
Name: Weekly_Sales, dtype: float64

In [30]:
#Variance in sales is below. 

var_str_sales=wmtdata.groupby('Store')['Weekly_Sales'].var()

In [31]:
var_str_sales

Store
1     2.433000e+10
2     5.649354e+10
3     2.145508e+09
4     7.086321e+10
5     1.424154e+09
6     4.516724e+10
7     1.267549e+10
8     1.129561e+10
9     4.764957e+09
10    9.136235e+10
11    2.750088e+10
12    1.936742e+10
13    7.049396e+10
14    1.008507e+11
15    1.452957e+10
16    7.356438e+09
17    1.258052e+10
18    3.120222e+10
19    3.675757e+10
20    7.612112e+10
21    1.657729e+10
22    2.600200e+10
23    6.239406e+10
24    2.813861e+10
25    1.276375e+10
26    1.219507e+10
27    5.756647e+10
28    3.303632e+10
29    9.824801e+09
30    5.202808e+08
31    1.583972e+10
32    1.904876e+10
33    5.823982e+08
34    1.094747e+10
35    4.462380e+10
36    3.687547e+09
37    4.768747e+08
38    1.829116e+09
39    4.729166e+10
40    1.416150e+10
41    3.530910e+10
42    2.526362e+09
43    1.648231e+09
44    6.131978e+08
45    1.694385e+10
Name: Weekly_Sales, dtype: float64

In [32]:
var_str_sales.sort_values(ascending=False)

Store
14    1.008507e+11
10    9.136235e+10
20    7.612112e+10
4     7.086321e+10
13    7.049396e+10
23    6.239406e+10
27    5.756647e+10
2     5.649354e+10
39    4.729166e+10
6     4.516724e+10
35    4.462380e+10
19    3.675757e+10
41    3.530910e+10
28    3.303632e+10
18    3.120222e+10
24    2.813861e+10
11    2.750088e+10
22    2.600200e+10
1     2.433000e+10
12    1.936742e+10
32    1.904876e+10
45    1.694385e+10
21    1.657729e+10
31    1.583972e+10
15    1.452957e+10
40    1.416150e+10
25    1.276375e+10
7     1.267549e+10
17    1.258052e+10
26    1.219507e+10
8     1.129561e+10
34    1.094747e+10
29    9.824801e+09
16    7.356438e+09
9     4.764957e+09
36    3.687547e+09
42    2.526362e+09
3     2.145508e+09
38    1.829116e+09
43    1.648231e+09
5     1.424154e+09
44    6.131978e+08
33    5.823982e+08
30    5.202808e+08
37    4.768747e+08
Name: Weekly_Sales, dtype: float64

In [33]:
coeff_of_var=mean_str_sales/std_str_sales

In [34]:
coeff_of_var

Store
1      9.970873
2      8.102160
3      8.694034
4      7.868902
5      8.426840
6      7.362531
7      5.068303
8      8.550456
9      7.880502
10     6.284032
11     8.179167
12     7.250300
13     7.546394
14     6.363884
15     5.171059
16     6.053978
17     7.966815
18     6.140790
19     7.536924
20     7.639263
21     5.872253
22     6.378248
23     5.564175
24     8.088169
25     6.255458
26     9.081773
27     7.398888
28     7.281744
29     5.442400
30    19.227797
31    11.091264
32     8.452336
33    10.767931
34     9.239989
35     4.353863
36     6.150859
37    23.761933
38     9.019129
39     6.670767
40     8.101772
41     6.748681
42    11.069866
43    15.599741
44    12.225939
45     6.038183
Name: Weekly_Sales, dtype: float64

In [35]:
#Which store/s had good quarterly growth in 3Q 2012
#all stores had negative (year over year) sales growth in 3Q2012: 
#The stores with the least negative sales growth were #44(-3.98%),#38(-5.82%),#39(-5.9%)

In [36]:
wmtdata

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,28-09-2012,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,05-10-2012,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,12-10-2012,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,19-10-2012,718125.53,0,56.47,3.969,192.330854,8.667


In [37]:
#1 first add column for Quarterly information. 
#Please notes these are claendar quarters.

In [38]:
wmtdata.Date=pd.to_datetime(wmtdata.Date)
wmtdata['Quarter']=pd.PeriodIndex(wmtdata.Date, freq='Q')


In [39]:
wmtdata.sort_values(['Store','Quarter'], inplace=True)

In [40]:
wmtdata.head(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1
21,1,2010-02-07,1492418.14,0,80.91,2.669,211.223533,7.787,2010Q1
30,1,2010-03-09,1540163.53,0,81.21,2.577,211.531248,7.787,2010Q1
34,1,2010-01-10,1453329.50,0,71.89,2.603,211.671989,7.838,2010Q1
43,1,2010-03-12,1548033.78,0,49.27,2.708,211.607193,7.838,2010Q1
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2010Q2


In [41]:
new_sales=wmtdata.loc[:,['Store','Quarter','Date','Weekly_Sales']]

In [42]:
new_sales.head(10)

,Store,Quarter,Date,Weekly_Sales
2,1,2010Q1,2010-02-19,1611968.17
3,1,2010Q1,2010-02-26,1409727.59
6,1,2010Q1,2010-03-19,1472515.79
7,1,2010Q1,2010-03-26,1404429.92
8,1,2010Q1,2010-02-04,1594968.28
21,1,2010Q1,2010-02-07,1492418.14
30,1,2010Q1,2010-03-09,1540163.53
34,1,2010Q1,2010-01-10,1453329.50
43,1,2010Q1,2010-03-12,1548033.78
0,1,2010Q2,2010-05-02,1643690.90


In [43]:
#sort by store and quarter
new_sales.sort_values(['Store','Quarter'], inplace=True)

In [44]:
new_sales.head()

,Store,Quarter,Date,Weekly_Sales
2,1,2010Q1,2010-02-19,1611968.17
3,1,2010Q1,2010-02-26,1409727.59
6,1,2010Q1,2010-03-19,1472515.79
7,1,2010Q1,2010-03-26,1404429.92
8,1,2010Q1,2010-02-04,1594968.28


In [45]:
qtr_sales=new_sales.loc[:,['Store','Quarter','Weekly_Sales']]

In [46]:
qtr_sales.head()

,Store,Quarter,Weekly_Sales
2,1,2010Q1,1611968.17
3,1,2010Q1,1409727.59
6,1,2010Q1,1472515.79
7,1,2010Q1,1404429.92
8,1,2010Q1,1594968.28


In [47]:
pct_change=qtr_sales.groupby(['Store','Quarter'])['Weekly_Sales'].sum().pct_change(4)

In [48]:
pct_change=pd.DataFrame(pct_change)

In [49]:
pct_change.sort_values(by='Quarter', inplace=True, ascending=False)

In [50]:
pct_change[40:91].sort_values(by='Weekly_Sales', ascending=False)

,,Weekly_Sales
Store,Quarter,
37,2012Q2,-0.003764
44,2012Q3,-0.039783
38,2012Q3,-0.058211
39,2012Q3,-0.059377
3,2012Q3,-0.070261
5,2012Q3,-0.082362
41,2012Q3,-0.083510
35,2012Q3,-0.090306
18,2012Q3,-0.099408


In [51]:
#Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in 
#non-holiday season for all stores together

In [52]:
#Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
#Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
#Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
#Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [53]:
wmtdata.head(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1
21,1,2010-02-07,1492418.14,0,80.91,2.669,211.223533,7.787,2010Q1
30,1,2010-03-09,1540163.53,0,81.21,2.577,211.531248,7.787,2010Q1
34,1,2010-01-10,1453329.50,0,71.89,2.603,211.671989,7.838,2010Q1
43,1,2010-03-12,1548033.78,0,49.27,2.708,211.607193,7.838,2010Q1
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2010Q2


In [54]:
#weekly mean sales for all stores in holiday season (Holiday_Flag=1)is:$1,122,888/-
#weekly mean sales for non holiday season (Holiday Flag=0) is: $1,041,256/-

In [55]:
wmtdata.groupby('Holiday_Flag')['Weekly_Sales'].mean()

Holiday_Flag
0    1.041256e+06
1    1.122888e+06
Name: Weekly_Sales, dtype: float64

In [56]:
holiday_data=wmtdata.loc[:,['Store','Date','Holiday_Flag','Weekly_Sales']]

In [57]:
holiday_data=pd.DataFrame(holiday_data)

In [58]:
holiday_data.head()
holiday_data.columns

Index(['Store', 'Date', 'Holiday_Flag', 'Weekly_Sales'], dtype='object')

In [59]:
holiday_data.sort_values(by='Holiday_Flag', ascending=False)

,Store,Date,Holiday_Flag,Weekly_Sales
4051,29,2010-12-31,1,465992.02
2852,20,2012-07-09,1,2080529.06
2137,15,2012-07-09,1,587259.82
3765,27,2010-12-31,1,1440963.00
1386,10,2011-12-30,1,1930690.37
...,...,...,...,...
2206,16,2011-08-04,0,439276.50
2193,16,2011-07-01,0,573545.96
2236,16,2011-04-11,0,511059.95
2223,16,2011-05-08,0,592981.33


In [60]:
holiday=holiday_data.loc[(holiday_data.Weekly_Sales >1041256)]

In [61]:
holiday

,Store,Date,Holiday_Flag,Weekly_Sales
2,1,2010-02-19,0,1611968.17
3,1,2010-02-26,0,1409727.59
6,1,2010-03-19,0,1472515.79
7,1,2010-03-26,0,1404429.92
8,1,2010-02-04,0,1594968.28
...,...,...,...,...
6337,45,2010-12-17,0,1123282.85
6338,45,2010-12-24,0,1682862.03
6386,45,2011-11-25,1,1170672.94
6389,45,2011-12-16,0,1078905.68


In [62]:
a=holiday.loc[(holiday.Holiday_Flag == 1)]

In [63]:
a

,Store,Date,Holiday_Flag,Weekly_Sales
1,1,2010-12-02,1,1641957.44
31,1,2010-10-09,1,1507460.69
42,1,2010-11-26,1,1955624.11
47,1,2010-12-31,1,1367320.01
83,1,2011-09-09,1,1540471.24
...,...,...,...,...
5819,41,2011-12-30,1,1264014.16
5855,41,2012-07-09,1,1392143.82
5825,41,2012-10-02,1,1238844.56
6334,45,2010-11-26,1,1182500.16


In [64]:
type(a)

pandas.core.frame.DataFrame

In [65]:
a.columns

Index(['Store', 'Date', 'Holiday_Flag', 'Weekly_Sales'], dtype='object')

In [66]:
a.groupby('Date').groups

{Timestamp('2010-10-09 00:00:00'): Int64Index([  31,  174,  460,  746, 1318, 1461, 1747, 1890, 2319, 2605, 2748,
             3177, 3320, 3606, 3749, 3892, 4321, 5465, 5751],
            dtype='int64'),
 Timestamp('2010-11-26 00:00:00'): Int64Index([  42,  185,  471,  757, 1043, 1329, 1472, 1615, 1758, 1901, 2044,
             2473, 2616, 2759, 2902, 3045, 3188, 3331, 3474, 3617, 3760, 3903,
             4332, 4475, 4761, 4904, 5476, 5619, 5762, 6334],
            dtype='int64'),
 Timestamp('2010-12-02 00:00:00'): Int64Index([   1,  144,  430,  716, 1288, 1431, 1574, 1717, 1860, 2432, 2575,
             2718, 3147, 3290, 3719, 3862, 4291, 4434, 4863, 5435, 5721],
            dtype='int64'),
 Timestamp('2010-12-31 00:00:00'): Int64Index([  47,  190,  476,  762, 1334, 1477, 1763, 1906, 2621, 2764, 3193,
             3336, 3765, 3908, 4337, 5481],
            dtype='int64'),
 Timestamp('2011-09-09 00:00:00'): Int64Index([  83,  226,  512,  798, 1370, 1513, 1799, 1942, 2371, 2657, 2800,
  

In [67]:
#comment on store data on a monthly and semester
#A: retail sales are impacted by seasonality and the economy. 2010 was a recession year and WMT's sales were lower compared to 2011. 
#on a monthly basis sales are impacted by holidays.
#Some of the highest months are:
#April/May(depending on when Easter falls): sales are impacted by the Easter Holiday and Memorial day. 
#Nov/Dec: sales are impacted by the Thanksgiving holiday, Black Friday after Thanksgiving and Christmas.
#July and Sept: July by 4th of july holiday and sept is impacted by back to school shopping in addition to holiday

In [68]:
wmtdata.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1


In [69]:
monthly_data=wmtdata.loc[:,['Store','Date','Weekly_Sales','Holiday_Flag']]

In [70]:
monthly_data.head()

,Store,Date,Weekly_Sales,Holiday_Flag
2,1,2010-02-19,1611968.17,0
3,1,2010-02-26,1409727.59,0
6,1,2010-03-19,1472515.79,0
7,1,2010-03-26,1404429.92,0
8,1,2010-02-04,1594968.28,0


In [71]:
monthly_data.Date=pd.to_datetime(monthly_data.Date)
monthly_data['Monthly']=pd.PeriodIndex(monthly_data.Date, freq='M')

In [72]:
monthly_data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Monthly
2,1,2010-02-19,1611968.17,0,2010-02
3,1,2010-02-26,1409727.59,0,2010-02
6,1,2010-03-19,1472515.79,0,2010-03
7,1,2010-03-26,1404429.92,0,2010-03
8,1,2010-02-04,1594968.28,0,2010-02


In [73]:
monthly_bystore=monthly_data.groupby(['Monthly'])['Weekly_Sales'].sum()

In [74]:
monthly_bystore=pd.DataFrame(monthly_bystore)

In [75]:
monthly_bystore.head(10)

,Weekly_Sales
Monthly,
2010-01,4.223988e+07
2010-02,1.915869e+08
2010-03,1.862262e+08
2010-04,1.838118e+08
2010-05,2.806119e+08
2010-06,1.424361e+08
2010-07,1.842664e+08
2010-08,1.845381e+08
2010-09,1.797041e+08


In [76]:
#group by semester or quarter
#A: Typically Q2 and Q4 are the strongest quarters for the company with 
#the holidays that i talked about above. 2012 Nov and Dec had lower sales. 
#in general WMT's sales are impacted by unemployemnt. 

In [77]:
wmtdata.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1


In [78]:
sem_bystore=wmtdata.groupby(['Quarter'])['Weekly_Sales'].sum()

In [79]:
sem_bystore=pd.DataFrame(sem_bystore)

sem_bystore.groupby(['Quarter'])['Weekly_Sales'].sum()

Quarter
2010Q1    4.200530e+08
2010Q2    6.068598e+08
2010Q3    5.485085e+08
2010Q4    7.134649e+08
2011Q1    5.360954e+08
2011Q2    6.019646e+08
2011Q3    6.435744e+08
2011Q4    6.665656e+08
2012Q1    5.457900e+08
2012Q2    6.172141e+08
2012Q3    5.559950e+08
2012Q4    2.811338e+08
Freq: Q-DEC, Name: Weekly_Sales, dtype: float64

In [81]:
sem_bystore.head(25)

,Weekly_Sales
Quarter,
2010Q1,4.200530e+08
2010Q2,6.068598e+08
2010Q3,5.485085e+08
2010Q4,7.134649e+08
2011Q1,5.360954e+08
2011Q2,6.019646e+08
2011Q3,6.435744e+08
2011Q4,6.665656e+08
2012Q1,5.457900e+08


In [82]:
sem_bystore=pd.DataFrame(sem_bystore)

In [83]:
sem_bystore.head(20)

,Weekly_Sales
Quarter,
2010Q1,4.200530e+08
2010Q2,6.068598e+08
2010Q3,5.485085e+08
2010Q4,7.134649e+08
2011Q1,5.360954e+08
2011Q2,6.019646e+08
2011Q3,6.435744e+08
2011Q4,6.665656e+08
2012Q1,5.457900e+08


In [84]:
monthly_bystore.tail(10)
#sales lower in Nov and Dec of 2012

,Weekly_Sales
Monthly,
2012-03,2.307397e+08
2012-04,1.825428e+08
2012-05,1.422830e+08
2012-06,2.923883e+08
2012-07,1.845865e+08
2012-08,1.916126e+08
2012-09,1.797959e+08
2012-10,1.880794e+08
2012-11,4.692588e+07


In [85]:
#store one data

In [86]:
store_one=wmtdata.loc[wmtdata.Store==1]

In [87]:
store_one.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1


In [88]:
store_one=store_one.set_index('Date')

In [89]:
store_one.head(25)

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
Date,,,,,,,,
2010-02-19,1,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
2010-02-26,1,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
2010-03-19,1,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
2010-03-26,1,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
2010-02-04,1,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1
2010-02-07,1,1492418.14,0,80.91,2.669,211.223533,7.787,2010Q1
2010-03-09,1,1540163.53,0,81.21,2.577,211.531248,7.787,2010Q1
2010-01-10,1,1453329.50,0,71.89,2.603,211.671989,7.838,2010Q1
2010-03-12,1,1548033.78,0,49.27,2.708,211.607193,7.838,2010Q1


In [90]:
#add a column to convert date into weekdays

In [91]:
from datetime import datetime
import calendar

In [92]:
store_one_mod=wmtdata.loc[wmtdata.Store==1]

In [93]:
type(store_one_mod)

pandas.core.frame.DataFrame

In [94]:
store_one_mod.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Quarter'],
      dtype='object')

In [95]:
store_one_mod.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1


In [96]:
store_one_mod["Day"]=store_one_mod['Date'].dt.strftime('%A')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
store_one_mod.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Quarter', 'Day'],
      dtype='object')

In [104]:
store_one_mod.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,Day
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1,Friday
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1,Friday
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1,Friday
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1,Friday
8,1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1,Thursday


In [132]:
#Linear regression model: The equation using Fuel prices,CPI, unemployment.
#The equation i get is:
# -3887095.77+ -23562.61963268(x1) + 14213.48484339 (x2)+ 67766.41155638 (x3)
#the model doesnt appear to be good with an accuracy score of 7.4%

In [106]:
import scipy
import pandas as pd
import numpy as np
from sklearn import metrics

In [107]:
from sklearn.linear_model import LinearRegression
lineReg=LinearRegression()

In [108]:
data=store_one

In [109]:
data.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
Date,,,,,,,,
2010-02-19,1,1611968.17,0,39.93,2.514,211.289143,8.106,2010Q1
2010-02-26,1,1409727.59,0,46.63,2.561,211.319643,8.106,2010Q1
2010-03-19,1,1472515.79,0,54.58,2.720,211.215635,8.106,2010Q1
2010-03-26,1,1404429.92,0,51.45,2.732,211.018042,8.106,2010Q1
2010-02-04,1,1594968.28,0,62.27,2.719,210.820450,7.808,2010Q1


In [110]:
data.columns

Index(['Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'Quarter'],
      dtype='object')

In [111]:
data.shape

(143, 8)

In [112]:
x_features=data.iloc[:,4:7]
y_target=data['Weekly_Sales']

In [113]:
x_features.head()

,Fuel_Price,CPI,Unemployment
Date,,,
2010-02-19,2.514,211.289143,8.106
2010-02-26,2.561,211.319643,8.106
2010-03-19,2.720,211.215635,8.106
2010-03-26,2.732,211.018042,8.106
2010-02-04,2.719,210.820450,7.808


In [114]:
y_target.head()

Date
2010-02-19    1611968.17
2010-02-26    1409727.59
2010-03-19    1472515.79
2010-03-26    1404429.92
2010-02-04    1594968.28
Name: Weekly_Sales, dtype: float64

In [115]:
y_target.shape

(143,)

In [116]:
data1=lineReg.fit(x_features,y_target)

In [117]:
print ('The intercept is %.2f'% lineReg.intercept_)

The intercept is -3887095.77


In [128]:
print ('The coefficient is %d'% len(lineReg.coef_))

The coefficient is 3


In [130]:
print('The coefficients are:',lineReg.coef_)

The coefficients are: [-23562.61963268  14213.48484339  67766.41155638]


In [119]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_features,y_target)

In [120]:
x_train.shape 

(107, 3)

In [121]:
x_test.shape

(36, 3)

In [122]:
y_train.shape 

(107,)

In [123]:
y_test.shape

(36,)

In [124]:
lineReg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [125]:
print('The MSE value is %.2f'% np.mean((lineReg.predict(x_test)-y_test)**2))

The MSE value is 24424705233.64


In [140]:
print('The variance score is %.3f'% lineReg.score(x_test, y_test))

The variance score is 0.074


In [137]:
accuracy = data1.score(x_test, y_test)
print("accuracy = ", accuracy*100, '%')

accuracy =  7.3615169076864495 %
